In [1]:
import os
from dotenv import load_dotenv

# 1. Carga las variables de entorno desde .env
load_dotenv()

True

In [2]:
from llm_structure import DocumentChat, DocumentChatOllama

In [3]:
system_prompt="Hello, your mission will be to help me extract information from the following documents your context is only this document, you never say nothing is not in the context. You will respond briefly and always in JSON {} format. Siempre responde en idioma Español."


In [4]:
system_prompt="You are an expert in extracting information from documents, your mission will be to give a document or book, extract relevant information, you will respond in JSON format with this scheme {result: 'the result'} you will not return anything else not even the markdown quotes, just the JSON."

In [5]:
#chat=DocumentChat("document.pdf","http://10.6.142.65:11434/v1","ollama","llama3.2:latest",system_prompt)

In [6]:
from enum import Enum
class TypeDocument(Enum):
    scientific_article="scientific article"
    book="scientific article"

In [7]:
from datetime import datetime

current_year = datetime.now().year
current_year

2025

In [8]:
from pydantic import BaseModel, Field
from typing import Annotated


class TestModel(BaseModel):
    document_title: str = Field(..., description="Who is the name of the document? ")
    book_or_scientific_articule: TypeDocument = Field(
        ..., description="Tell me if the document is a book or a scientific_article "
    )

    year_publish: Annotated[
        int,
        Field(
            description=f"Year the document was published. Must be > 1990 and <= {current_year} .",
            gt=1990,
            le=current_year,
        ),
    ]
    
    

In [9]:
TestModel.model_json_schema()

{'$defs': {'TypeDocument': {'enum': ['scientific article',
    'scientific article'],
   'title': 'TypeDocument',
   'type': 'string'}},
 'properties': {'document_title': {'description': 'Who is the name of the document? ',
   'title': 'Document Title',
   'type': 'string'},
  'book_or_scientific_articule': {'$ref': '#/$defs/TypeDocument',
   'description': 'Tell me if the document is a book or a scientific_article '},
  'year_publish': {'description': 'Year the document was published. Must be > 1990 and <= 2025 .',
   'exclusiveMinimum': 1990,
   'maximum': 2025,
   'title': 'Year Publish',
   'type': 'integer'}},
 'required': ['document_title', 'book_or_scientific_articule', 'year_publish'],
 'title': 'TestModel',
 'type': 'object'}

In [10]:
from markitdown import MarkItDown

md = MarkItDown(enable_plugins=True) # Set to True to enable plugins
result = md.convert("document.pdf")
print(result.text_content)
context=result

Artículo Original / Original Research

Ciencias Matemáticas, Vol. 37, No. 1, 2024, Pag. 1-7
Recibido (Received): 26-05-2024, Revisado (Reviewed): 04-07-2024
Aceptado (Accepted): 22-10-2024, En línea (Online): 01-12-2024

Ejercicios de cálculo, de orientación y divergentes
utilizados en los contenidos de Geometría Analítica
Exercises in calculus, orientation, and divergence
used in the contents of Analytic Geometry

Roxana Cabrera Puig1* , Alicia Robbio Marrero2

Resumen En la metodología de la enseñanza de la Matemática, una de las líneas directrices fundamentales
es el tratamiento de los contenidos geométricos para fijar las ideas esenciales de los conceptos y que se puedan
demostrar, inferir y establecer analogías, lo que favorece la resolución de problemas matemáticos donde los
medios de cómputo sirvan de apoyo para una mejor visualización del contenido que se aborda. Es por ello que,
desde las clases se recomienda impulsar el empleo del Geogebra como software dinámico para la compr

In [16]:
from openai import OpenAI
client = OpenAI(
    base_url='http://10.6.141.103:8080/v1',
    api_key='ollama'  # requerido, pero no se usa realmente
)

response = client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",  # o el nombre de tu modelo local
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{context} Given this document, it responds in a JSON in reference to the name of the document, whether it is a scientific article and the publication date. Bajo este formato {TestModel.model_json_schema()} responde solo en json y no con comillas de markdown. The JSON is: "}
    ],
    temperature=0,
    response_format={
                "type": "json_object",
                "schema": TestModel.model_json_schema(),
            }
)

print(response.choices[0].message.content)

{
  "result": "Ejercicios de cálculo, de orientación y divergentes utilizados en los contenidos de Geometría Analítica"
}


In [15]:
chat=DocumentChat("document.pdf","http://10.6.141.103:8080/v1","ollama","gemma-3-4b-it",system_prompt)

In [13]:
query=chat.ask_document

## Titulo del documento

In [19]:
query("Give me the title of the document." )

'Ejercicios de cálculo, de orientación y divergentes utilizados en los contenidos de Geometría Analítica'

## Autores

In [ ]:
query("Dime los nombres de los autores? separalos por , y espacio ejemplo: Francisco Vicente Suarez Bellon, Rodrigo Peralta, Rocio Bravo")

## Decir si es un articulo o una publicacion 

In [ ]:
def query_grammar(query,prepositions:list[str]):
    root_grammar=f"""
    root ::= response
    response ::= {' | '.join(f'"{item}"' for item in prepositions)}
    """
    return chat.ask_with_gbnf_grammar(query,root_grammar)

In [ ]:
query_grammar("Es documento es un articulo cientifico o es un libro?",["Libro","Articulo"])

In [ ]:
query("En que año se publico el articulo? devuelve un numero solamente y que sea una año valido entre 1900-2025")

In [ ]:
query("Cual es el nombre de la editorial de la revista que lo publico?")

In [ ]:
query("Cual es el pais de publicacion?")

In [ ]:
query("Cual es la url de publicación?")

In [ ]:
query("Cual es el orcid de cada autor?")

In [ ]:
query("Cual es el doi del documento?")

In [ ]:
query("Cual es el tipo de medio de divulgación? , responde entre revista o libro")

In [ ]:
query("Desde que area se reporta?")

In [ ]:
query_boolean=chat.boolean_ask_document


In [ ]:
query_boolean("El documento tributa a algun proyecto de ciencia o investigación? no vale la revista tipo")

In [ ]:
query("Cual es el proyecto de ciencia e investigacion al que tributa? No vale la revista tipo ")

In [ ]:
query("Cual es el tipo de ciencia de este articulo? dentro del area de las matematicas?")

In [ ]:
query("Cuales son las palabras clave?")

In [ ]:
query("En base a las palabras clave (exercises, Geogebra, geometry) que tipo de disciplina es dentro de las matematicas?")